In [42]:
import os
import pandas as pd
from sqlalchemy import create_engine

def get_engine(user, password, host, port, database):
    connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)
    return engine


In [43]:
def rentals_month(engine, month, year):
    query = f"""
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    return pd.read_sql(query, engine)


In [44]:
def rental_count_month(df, month, year):
    rentals_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rentals_count


In [45]:
def compare_rentals(df1, df2):
    comparison_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    comparison_df['difference'] = comparison_df.iloc[:, 1] - comparison_df.iloc[:, 2]
    return comparison_df


In [46]:
user = '*****'
password = '*******' 
host = '*****'
port = '****' 
database = 'sakila'

In [47]:
engine = get_engine(user, password, host, port, database)

In [48]:
rentals_may_2005 = rentals_month(engine, 5, 2005)
rentals_june_2005 = rentals_month(engine, 6, 2005)

In [49]:
may_rentals_count = rental_count_month(rentals_may_2005, 5, 2005)
june_rentals_count = rental_count_month(rentals_june_2005, 6, 2005)

In [50]:
comparison_df = compare_rentals(may_rentals_count, june_rentals_count)

In [51]:
comparison_df


,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,4,0.0,6.0,-6.0
4,5,3.0,5.0,-2.0
...,...,...,...,...
593,595,1.0,2.0,-1.0
594,596,6.0,2.0,4.0
595,597,2.0,3.0,-1.0
596,598,0.0,1.0,-1.0
